In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn

from plaid.vqvae import TransformerVQVAE
from plaid.proteins import LatentToSequence, LatentToStructure

In [5]:
file = "/shared/amyxlu/plaid/checkpoints/2024-01-11T15-45-26/last.ckpt"
ckpt = torch.load(file)
ckpt.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])

In [6]:
# ckpt = torch.load(file)
model = TransformerVQVAE.load_from_checkpoint(file, vqvae_h_dim=512, vqvae_res_h_dim=512)

RuntimeError: Error(s) in loading state_dict for TransformerVQVAE:
	Missing key(s) in state_dict: "vqvae_encoder.conv_stack.5.stack.3.res_block.1.weight", "vqvae_encoder.conv_stack.5.stack.3.res_block.3.weight", "vqvae_encoder.conv_stack.5.stack.4.res_block.1.weight", "vqvae_encoder.conv_stack.5.stack.4.res_block.3.weight", "vqvae_encoder.conv_stack.5.stack.5.res_block.1.weight", "vqvae_encoder.conv_stack.5.stack.5.res_block.3.weight", "vqvae_encoder.conv_stack.5.stack.6.res_block.1.weight", "vqvae_encoder.conv_stack.5.stack.6.res_block.3.weight", "vqvae_encoder.conv_stack.5.stack.7.res_block.1.weight", "vqvae_encoder.conv_stack.5.stack.7.res_block.3.weight", "vqvae_encoder.conv_stack.5.stack.8.res_block.1.weight", "vqvae_encoder.conv_stack.5.stack.8.res_block.3.weight", "vqvae_encoder.conv_stack.5.stack.9.res_block.1.weight", "vqvae_encoder.conv_stack.5.stack.9.res_block.3.weight", "vqvae_encoder.conv_stack.5.stack.10.res_block.1.weight", "vqvae_encoder.conv_stack.5.stack.10.res_block

In [ ]:
from plaid.esmfold import esmfold_v1
esmfold = esmfold_v1()

In [10]:
device = torch.device("cuda:0")
latent_to_structure = LatentToStructure(device, esmfold)
latent_to_sequence = LatentToSequence(device)

In [12]:
from plaid.datasets import CATHShardedDataModule

dm = CATHShardedDataModule(
    shard_dir ="/shared/amyxlu/data/cath/shards",
    header_to_sequence_file="/shared/amyxlu/data/cath/sequences.pkl",
    seq_len=64,
    dtype="fp32",
)
dm.setup()
loader = dm.val_dataloader()
batch = next(iter(loader))

In [15]:
model = model.to(device)
model = model.eval().requires_grad_(False)

x = batch[0].to(device)
output = model(x)

In [21]:
x_hat = output[1]
torch.mean((x_hat - x) ** 2)

tensor(5212.0371, device='cuda:0')

In [27]:
sequences = [s[:64] for s in batch[-1]]
pred_pdb = latent_to_structure.to_structure(x_hat, sequences)

(Generating structure from latents..): 100%|██████████| 1/1 [11:18<00:00, 678.82s/it]
